In [1]:
import os
import numpy as np
import pandas as pd

def align_sequences(X, y):
    if X.shape[0] > y.shape[0]:
        X = X[:y.shape[0], :]
    elif y.shape[0] > X.shape[0]:
        delta = y.shape[0] - X.shape[0]
        cut_start = delta // 2
        cut_end = delta - cut_start
        y = y[cut_start: -cut_end if cut_end > 0 else None, :]
    return X, y

def load_trial_data(imu_trial_folder, angles_file, window_size):
    imu_data = []

    # Load all IMU CSVs inside trial folder
    for fname in sorted(os.listdir(imu_trial_folder)):
        if fname.endswith(".csv"):
            df = pd.read_csv(os.path.join(imu_trial_folder, fname))
            imu_data.append(df.values)

    if not imu_data:
        raise RuntimeError(f"No IMU files found in {imu_trial_folder}")

    X = np.concatenate(imu_data, axis=1)

    y = pd.read_csv(angles_file).values
    X, y = align_sequences(X, y)

    X_seq, y_seq = [], []
    for i in range(len(X) - window_size + 1):
        X_seq.append(X[i:i + window_size])
        y_seq.append(y[i + window_size - 1])

    return np.array(X_seq), np.array(y_seq)

In [2]:
import importlib.util

torch_spec = importlib.util.find_spec("torch")
print(torch_spec)

ModuleSpec(name='torch', loader=<_frozen_importlib_external.SourceFileLoader object at 0x00000208FEC37450>, origin='d:\\Misk\\masters\\.venv\\Lib\\site-packages\\torch\\__init__.py', submodule_search_locations=['d:\\Misk\\masters\\.venv\\Lib\\site-packages\\torch'])


In [3]:
from torch.utils.data import Dataset
import torch

class IMUMultiTrialDataset(Dataset):
    def __init__(self, imu_data_root, angles_dir, trial_ids, window_size):
        self.X, self.y = [], []
        for trial_id in trial_ids:
            try:
                trial_name = f"trial_{trial_id}"
                imu_trial_folder = os.path.join(imu_data_root, trial_name)
                angles_file = os.path.join(angles_dir, f"{trial_name}.csv")
                X_seq, y_seq = load_trial_data(imu_trial_folder, angles_file, window_size)
                self.X.append(X_seq)
                self.y.append(y_seq)
            except Exception as e:
                print(f"Skipping {trial_name}: {e}")

        self.X = np.concatenate(self.X)
        self.y = np.concatenate(self.y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.y[idx], dtype=torch.float32)
        )

imu_data_root = os.path.abspath("dataset/imu_data")
angles_dir = os.path.abspath("dataset/angles")
window_size = 50

# Define splits by trial ID
train_ids = [1, 2]
val_ids   = [3]
test_ids  = [4]

train_dataset = IMUMultiTrialDataset(imu_data_root, angles_dir, train_ids, window_size)
val_dataset   = IMUMultiTrialDataset(imu_data_root, angles_dir, val_ids, window_size)
test_dataset  = IMUMultiTrialDataset(imu_data_root, angles_dir, test_ids, window_size)

# Sanity check
x0, y0 = train_dataset[0]
print("Sample input shape:", x0.shape)
print("Sample target shape:", y0.shape)

Skipping trial_1: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 10178 and the array at index 1 has size 4911
Skipping trial_2: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 3820 and the array at index 1 has size 7259


ValueError: need at least one array to concatenate